# Gekoppelde tanks met zoutoplossing

De Differentiaal vergelijkingen voor de zoutconcentratie van tank A ( x ) en B (y) zijn: <br>
De differentiaal vergelijking voor tank A:
$$\frac{dx}{dt} = 0.01y - 0.0697x +1.11636$$
 
De differentiaalvergelijking van tank B: 
$$\frac{d
y}{dt} = -0.03y + 0.0291x + -0.03492 $$

In [57]:
# importeer benodigde libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact #  interactieve visualisatie

In [8]:
totaltime = 200 # totale  tjdin minuten
Czout = 0.2   # zoutconcentratie  in kg/L

Er wordt gebruik gemaakt van de 2 differentiaalvergelijkingen hieronder om de Forward Euler en de Heuns methode toe te passen: <br>
x (zoutconcentratie in tank A) = 1.2  +(1/100 * (3/100(x +1.2)+y)) - (3/100(x+1.2)) - (4/100(x+1.2)) <br>

y (zoutconcentratie in tank B) = (3/100(x+1.2)) - (1/100 * (3/100(x +1.2)+y)) - (2/100(y + (3/100(x +1.2)))

## Forward Euler

Er wordt eerst Forward Euler method toegepast.

In [19]:
def forwardeuler(h=1):
    xin = 6 *  Czout # Er wordt elke minuut 6L zoutoplossing van 0.2kg/L in  de tank toegevoegd. Dit is gelijk aan 6*0.2 = 1.2 k Zout
    t = h * np.array(range(totaltime+1)) # totale tijd in min
    x = np.zeros(totaltime+1) # zoutgehalte in kg van tank A
    y = np.zeros(totaltime+1) # zoutgehalte in kg van tank B 
    
    # aantal kg zout bij t=0
    x[0] = 0
    y[0] = 20
    
    for step in range(totaltime):
        # De differentiaal vergelijking wordt opesplits zodat er is meer overzichtelijkheid zit in de code.
        xovery = (3/100) * (x[step] + xin) # (3/100(x +1.2)
        yoverx = (1/100) * (y[step] + xovery) # 1/100 * ((3/100(x +1.2))+y)
        xuit   = (4/100) * (x[step]  + xin)  # 4/100(x+1.2)
        yuit   = (2/100) * (y[step] + xovery) # 2/100(y + (3/100(x +1.2)
        #Forward Euler method
        x[step+1] = x[step] + h * (xin + yoverx - xovery - xuit)
        y[step+1] = y[step] + h * (xovery - yoverx - yuit)
    return t,x,y

    

In [52]:
#Plot van de Forward Euler method
@interact
def plotten(h=1):
    t,conc_tank1, conc_tank2 = forwardeuler(h)
    # plot the first tank
    plt.plot(t, conc_tank1/100, label="tank1")

    # plot the second tank
    plt.plot(t, conc_tank2/100, label="tank2")

    # setting the plot
    plt.title("Salt concentrations over time")
    plt.xlabel("Time (minutes)")
    plt.ylabel("salt concentration (kg/L)")

    # show the plot
    plt.legend()
    plt.show()

interactive(children=(IntSlider(value=1, description='h', max=3, min=-1), Output()), _dom_classes=('widget-int…

## Heuns methode met adaptieve stapgrootte
Er wordt ook Heuns methode geimplementeerd om  nauwkeurige resultaten te krijgen.

In [47]:
def heuns(h=1):
    xin = 6 *  Czout # Er wordt elke minuut 6L zoutoplossing van 0.2kg/L in  de tank toegevoegd. Dit is gelijk aan 6*0.2 = 1.2 k Zout
    x = np.zeros(1) # zoutgehalte in kg van tank A
    y = np.zeros(1) # zoutgehalte in kg van tank B
    current_time = 0 #minuten
    tolerance = 5e5
    
    x[0] = 0
    y[0] = 20
    while current_time < totaltime:
        xovery = (3/100) * (x + xin)
        yoverx = (1/100) * (y + xovery)
        xuit   = (4/100) * (x  + xin)
        yuit   = (2/100) * (y + xovery)
        
        # Heun's methode 
        xe = x + h * (xin + yoverx - xovery - xuit)
        ye = y + h * (xovery - yoverx - yuit)
        
        xy3 = (3/100) * (xe + xin)
        yx3 = (1/100) * (ye + xy3)
        xu3 = (4/100) * (xe  + xin)
        yu3 = (2/100) * (ye + xy3)
        
        x = x + (h/2) *((xin + yoverx - xovery - xuit) + (xin + yx3 - xy3 -xu3))
        y = y + (h/2) *((xovery - yoverx - yuit) + (xy3 - yx3 - yu3))
        
        current_time += h
        LTE = np.linalg.norm(xe - x) + (totaltime*np.linalg.norm(ye - y))
        h_new = h * np.sqrt(tolerance/LTE)#h * sqrt(tolerance/LTE)
        h = h_new
    return x,y
    

## Backward Euler method

er is well geen duidelijk instabliteit te zien in deze probleem , maar er is toch gekozen om de backward euler method te implementeren. <br>
Voor deze implementatie  iser gebruikt gemaakt van de uitgewerkte differentialverelijkingen: <br>
x'(t) = 0.01𝑦−0.0697𝑥+1.11636<br>
y'(t) = −0.03𝑦+0.0291𝑥+−0.03492



In [48]:
 def backwardeuler(h=1):
    t = h * np.array(range(totaltime+1)) # totale tijd in min
    x = np.zeros(totaltime+1) # zoutgehalte in kg van tank A
    y = np.zeros(totaltime+1) # zoutgehalte in kg van tank B
    
    for step in range(totaltime):
        x[step+1] = (x[step] + (0.01*h * y[step+1]) + (1.11636*h))/(1+ (0.0697*h))
        y[step+1] = (y[step] + (0.0291*h*x[step+1]) - (0.03492*h))/(1 + (0.03*h))
    return t,x,y

In [56]:
#Plot van de Backward Euler method
@interact
def plotten(h=5):
    t,conc_tank1, conc_tank2 = backwardeuler(h)
    # plot  tank A
    plt.plot(t, conc_tank1/100, label="tank1")

    # plot  tank B
    plt.plot(t, conc_tank2/100, label="tank2")

    plt.title("Zoutconcentratie over een bepaalde tijd")
    plt.xlabel("Tijd in minuten")
    plt.ylabel("Zoutconcentratie in kg/L")

    # show the plot
    plt.legend()
    plt.show()

interactive(children=(IntSlider(value=5, description='h', max=15, min=-5), Output()), _dom_classes=('widget-in…